# Bayesian estimation

This is the source codes for the example discussed in Sec. III E in Ref. [1].

The Hamiltonian of a qubit system is
\begin{align}
H=\frac{B\omega_0}{2}(\sigma_1\cos{x}+\sigma_3\sin{x}),
\end{align}

where $B$ is the magnetic field in the XZ plane, $x$ is the unknown parameter and $\sigma_{1}$, $\sigma_{3}$ are the Pauli matrices.

The probe state is taken as $|\pm\rangle$. The measurement is 
$\{|\!+\rangle\langle+\!|,|\!-\rangle\langle-\!|\}$. Here $|\pm\rangle:=\frac{1}{\sqrt{2}}(|0\rangle\pm|1\rangle)$ with $|0\rangle$ $(|1\rangle)$ the eigenstate of $\sigma_3$ with respect to the eigenvalue $1$ $(-1)$. In this example, the prior distribution $p(x)$ is uniform on $[0, \pi/2]$.

[1] M. Zhang et al., QuanEstimation: an open-source toolkit for quantum parameter estimation,
arXiv:2022.xxxx.

In [1]:
from quanestimation import *
import numpy as np
import random

# initial state
rho0 = 0.5*np.array([[1., 1.], [1., 1.]])
# free Hamiltonian
B, omega0 = np.pi/2.0, 1.0
sx = np.array([[0., 1.], [1., 0.]])
sy = np.array([[0., -1.j], [1.j, 0.]]) 
sz = np.array([[1., 0.], [0., -1.]])
H0_func = lambda x: 0.5*B*omega0*(sx*np.cos(x)+sz*np.sin(x))
# derivative of the free Hamiltonian on x
dH_func = lambda x: [0.5*B*omega0*(-sx*np.sin(x)+sz*np.cos(x))]
# measurement
M1 = 0.5*np.array([[1., 1.], [1., 1.]])
M2 = 0.5*np.array([[1.,-1.], [-1., 1.]])
M = [M1, M2]
# prior distribution
x = np.linspace(0., 0.5*np.pi, 1000)
p = (1.0/(x[-1]-x[0]))*np.ones(len(x))
# time length for the evolution
tspan = np.linspace(0., 1., 1000)
# dynamics
rho = [np.zeros((len(rho0), len(rho0)), dtype=np.complex128) \
       for i in range(len(x))]
for i in range(len(x)):
    H0 = H0_func(x[i])
    dH = dH_func(x[i])
    dynamics = Lindblad(tspan, rho0, H0, dH)
    rho_tp, drho_tp = dynamics.expm()
    rho[i] = rho_tp[-1]

Generation of the experimental results

In [2]:
random.seed(1234)
y = [0 for i in range(500)]
res_rand = random.sample(range(0,len(y)), 125)
for i in range(len(res_rand)):
    y[res_rand[i]] = 1

Maximum a posteriori estimation

In [4]:
pout, xout = Bayes([x], p, rho, y, M=M, estimator="MAP", savefile=False)

Maximum likelihood estimation

In [ ]:
Lout, xout = MLE([x], rho, y, M=M, savefile=False)